In [15]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import json
from PIL import Image
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import random

import tensorflow as tf
# Needed for VGG16
tf.config.run_functions_eagerly(True)

from sklearn.datasets import load_files   
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.saving import load_model
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Loading Data (Numpy)

In [2]:
with open('./Data/X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('./Data/y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('./Data/X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('./Data/y_test.npy', 'rb') as f:
    y_test = np.load(f)

In [3]:
dimensions = (200, 200)
num_classes = 5

# VGG16

In [4]:
# Necessary for ResNet Dense Layer
y_train_eye = np.eye(num_classes)[y_train.reshape(-1)]
y_test_eye = np.eye(num_classes)[y_test.reshape(-1)]

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16

In [10]:
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = VGG16(weights= None, include_top=False, input_shape=(dimensions[0], dimensions[1], 3))

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [12]:
from keras.optimizers import SGD, Adam
# sgd = SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train_ros, y_train_ros, epochs=50, batch_size=64)

Epoch 1/25
132/132 [==============================] - 1180s 9s/step - loss: 2.0631 - accuracy: 0.3792
Epoch 2/25
132/132 [==============================] - 1125s 9s/step - loss: 1.0150 - accuracy: 0.6178
Epoch 3/25
132/132 [==============================] - 1115s 8s/step - loss: 0.7566 - accuracy: 0.6878
Epoch 4/25
132/132 [==============================] - 1116s 8s/step - loss: 0.5423 - accuracy: 0.7667
Epoch 5/25
132/132 [==============================] - 1146s 9s/step - loss: 0.4451 - accuracy: 0.8000
Epoch 6/25
132/132 [==============================] - 1203s 9s/step - loss: 0.3526 - accuracy: 0.8447
Epoch 7/25
132/132 [==============================] - 1156s 9s/step - loss: 0.2993 - accuracy: 0.8807
Epoch 8/25
132/132 [==============================] - 1153s 9s/step - loss: 0.2514 - accuracy: 0.8953
Epoch 9/25
132/132 [==============================] - 1194s 9s/step - loss: 0.1634 - accuracy: 0.9361
Epoch 10/25
132/132 [==============================] - 1108s 8s/step - loss: 0.173

In [15]:
model.save('./ModelSaves/VGG16v1')

INFO:tensorflow:Assets written to: ./ModelSaves/VGG16v1\assets


INFO:tensorflow:Assets written to: ./ModelSaves/VGG16v1\assets


In [6]:
model = load_model('./ModelSaves/VGG16v1')

In [7]:
preds = model.evaluate(X_test, y_test_eye)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

26/26 [==============================] - 131s 5s/step - loss: 2.4212 - accuracy: 0.6185
Loss = 2.4211628437042236
Test Accuracy = 0.6184689998626709


In [ ]:
preds = model(X_test)

In [ ]:
preds[0:5]

In [ ]:
np.sum(np.argmax(preds, axis=1) == np.argmax(y_test_norm, axis=1)) / len(y_test)

In [ ]:
np.unique(np.argmax(preds, axis=1), return_counts=True)

In [ ]:
emotions = ["Happiness", "Sadness", "Fear", "Disgust", "Anger", "Surprise"]

emotion_presence = set()

rands = []

shuffle_indices = shuffle(range(len(preds)))

for i in shuffle_indices:
    if emotions[classes[i]] not in emotion_presence and np.argmax(preds[i]) == classes[i]:
        rands.append(i)
        emotion_presence.add(emotions[classes[i]])

print(len(rands))

while len(rands) < 5:
    pos = random.randint(0, len(test_indices) - 1)
    if emotions[classes[test_indices[pos]]] not in emotion_presence:
        rands.append(pos)
        emotion_presence.add(emotions[classes[test_indices[pos]]])

# actuals = [test_indices[pos] for pos in rands]

for i in range(len(actuals)):
    print('========================================================')
    print(book_links[actuals[i]])
    display(Image.fromarray((X_test[rands[i]] * 255).astype(np.uint8)))
    print(emotions[np.argmax(preds[i])])
    im = Image.open(directory + str(actuals[i]) + ".jpg")
    im.thumbnail(dimensions, Image.ANTIALIAS)
    display(im)
    print(emotions[classes[actuals[i]]])
    print('========================================================')

In [ ]:
scores = {}
for i in range(len(y_test)):
    real_class = y_test[i]
    pred_class = np.argmax(preds[i])
    if real_class in scores:
        scores[real_class] = [scores[real_class][0] + int(real_class == pred_class), scores[real_class][1] + 1]
    else:
        scores[real_class] = [int(real_class == pred_class), 1]

In [ ]:
for key in list(scores.keys()):
    print(key, scores[key], scores[key][0] / scores[key][1])

# Oversampling

In [4]:
ros = RandomOverSampler()

X_train_ros, y_train_ros = ros.fit_resample(
    X_train.reshape((-1, X_train.shape[1] * X_train.shape[2] * X_train.shape[3])),
    y_train
)

X_train_ros = X_train_ros.reshape((-1, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
X_train_ros.shape, y_train_ros.shape

((8305, 200, 200, 3), (8305,))

In [8]:
# Necessary for ResNet Dense Layer
y_train_ros_eye = np.eye(num_classes)[y_train_ros.reshape(-1)]
y_test_eye = np.eye(num_classes)[y_test.reshape(-1)]

In [16]:
from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.resnet50 import ResNet50
base_model = VGG16(weights= None, include_top=False, input_shape=(dimensions[0], dimensions[1], 3))
# base_model = ResNet50(weights= None, include_top=False, input_shape=(dimensions[0], dimensions[1], 3))

# Required for VGG16 in thi cae
tf.config.run_functions_eagerly(True)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

from keras.optimizers import SGD, Adam
# sgd = SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_ros, y_train_ros_eye, epochs=50, batch_size=64)

C:\Users\Brian\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/50
130/130 [==============================] - 5773s 44s/step - loss: 1.6095 - accuracy: 0.2190
Epoch 2/50
130/130 [==============================] - 5705s 44s/step - loss: 1.5987 - accuracy: 0.2226
Epoch 3/50
130/130 [==============================] - 5719s 44s/step - loss: 1.5221 - accuracy: 0.2848
Epoch 4/50
130/130 [==============================] - 5759s 44s/step - loss: 1.2613 - accuracy: 0.3966
Epoch 5/50
130/130 [==============================] - 5802s 45s/step - loss: 0.9182 - accuracy: 0.5440
Epoch 6/50
130/130 [==============================] - 53333s 413s/step - loss: 0.7470 - accuracy: 0.6033
Epoch 7/50
130/130 [==============================] - 5878s 45s/step - loss: 0.6820 - accuracy: 0.6148
Epoch 8/50
130/130 [==============================] - 5822s 45s/step - loss: 0.6874 - accuracy: 0.6194
Epoch 9/50
130/130 [==============================] - 5818s 45s/step - loss: 0.6486 - accuracy: 0.6466
Epoch 10/50
130/130 [==============================] - 5840s 45s/step -

In [ ]:
preds = model.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
model.save('./ModelSaves/VGG16_oversampling_v1')

In [8]:
preds = model.evaluate(X_test, y_test_eye)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

26/26 [==============================] - 141s 5s/step - loss: 2.4212 - accuracy: 0.6185
Loss = 2.4211628437042236
Test Accuracy = 0.6184689998626709


In [ ]:
preds = model(X_test)

In [ ]:
preds[0:5]

In [ ]:
np.sum(np.argmax(preds, axis=1) == np.argmax(y_test_norm, axis=1)) / len(y_test)

In [ ]:
np.unique(np.argmax(preds, axis=1), return_counts=True)

In [ ]:
emotions = ["Happiness", "Sadness", "Fear", "Disgust", "Anger", "Surprise"]

emotion_presence = set()

rands = []

shuffle_indices = shuffle(range(len(preds)))

for i in shuffle_indices:
    if emotions[classes[i]] not in emotion_presence and np.argmax(preds[i]) == classes[i]:
        rands.append(i)
        emotion_presence.add(emotions[classes[i]])

print(len(rands))

while len(rands) < 5:
    pos = random.randint(0, len(test_indices) - 1)
    if emotions[classes[test_indices[pos]]] not in emotion_presence:
        rands.append(pos)
        emotion_presence.add(emotions[classes[test_indices[pos]]])

# actuals = [test_indices[pos] for pos in rands]

for i in range(len(actuals)):
    print('========================================================')
    print(book_links[actuals[i]])
    display(Image.fromarray((X_test[rands[i]] * 255).astype(np.uint8)))
    print(emotions[np.argmax(preds[i])])
    im = Image.open(directory + str(actuals[i]) + ".jpg")
    im.thumbnail(dimensions, Image.ANTIALIAS)
    display(im)
    print(emotions[classes[actuals[i]]])
    print('========================================================')

In [ ]:
scores = {}
for i in range(len(y_test)):
    real_class = y_test[i]
    pred_class = np.argmax(preds[i])
    if real_class in scores:
        scores[real_class] = [scores[real_class][0] + int(real_class == pred_class), scores[real_class][1] + 1]
    else:
        scores[real_class] = [int(real_class == pred_class), 1]

In [ ]:
for key in list(scores.keys()):
    print(key, scores[key], scores[key][0] / scores[key][1])